# Mosaic and Backfill FSim Outputs for All WCS Landscapes at 270m
#### Charlie Curtin 12/09/24

The goal of this notebook is to take the 270m FSim hazard outputs by pyrome and process the results to create one mosaic masked to the WCS landscapes. The FSim outputs are 270m, multi-band rasters by pyrome. The band order is as follows:

1. burn probability (BP)
2. flame length probability 1 (FLP1)
3. flame length probability 2 (FLP2)
4. flame length probability 3 (FLP3)
5. flame length probability 4 (FLP4)
6. flame length probability 5 (FLP5)
7. flame length probability 6 (FLP6)
8. crown flame length (CFL, not used in this analysis)

*This notebook is adapted from Mitch Lazarz*

### Setup

In [8]:
#%reset -f

##-----load libraries
import arcpy
import os
from itertools import repeat
from arcpy import env
from arcpy.sa import *

##------set up folders and directories
# define working directory- folder where data is housed
wd = r"C:\Users\Charlie\Desktop\ArcGIS\data"
# project path
proj_path = r"C:\Users\Charlie\Desktop\ArcGIS\FSim_Mosaic_Backfill_Clip_Landscape"

##------paths to data inputs
#--final extent
# wildfire crisis strategy landscapes buffered by 60km
wcs_buffer = os.path.join(wd, r"FSim_mosaic_data\FSim_mosaic_data\AnalysisArea\AnalysisArea_Buffered_60km.shp")

#--mosaic inputs
# 270m FSim multi-band raster outputs by pyrome
fsim270 = os.path.join(wd, r"FSim_mosaic_data\FSim_mosaic_data\4_FSim_Outputs\Jan2021\Outputs_by_Pyrome_270m")
# pyrome lookup table with the number of FSim iterations
iteration_lut_path = os.path.join(wd, r"FSim_mosaic_data\FSim_mosaic_data\3_FSim_Inputs\full_lookup.csv")

##--backfill inputs
# mosaicked fuel rasters
fm40 = os.path.join(wd, r"FSim_mosaic_data\FSim_mosaic_data\2_Fuelscapes\Output\Fuels_Mosaics_270m\Fuels_270m\S3_40_270m.tif")
cc = os.path.join(wd, r"FSim_mosaic_data\FSim_mosaic_data\2_Fuelscapes\Output\Fuels_Mosaics_270m\Fuels_270m\S3_CC_270m.tif")
cbh = os.path.join(wd, r"FSim_mosaic_data\FSim_mosaic_data\2_Fuelscapes\Output\Fuels_Mosaics_270m\Fuels_270m\S3_CBH_270m.tif")

# 270m pyrome boundaries raster
pyromes = os.path.join(wd, r"FSim_mosaic_data\FSim_mosaic_data\4_FSim_Outputs\Jan2021\Mosaic_Backfill_Inputs\Extent_CONUS_PYROME_270m\Extent_CONUS_PYROME_270m.tif")
# 270m snap raster
snap = os.path.join(wd, r"FSim_mosaic_data\FSim_mosaic_data\4_FSim_Outputs\Jan2021\Mosaic_Backfill_Inputs\SnapRasters\Extent_CONUS_270m.tif")

##-----set environments
#arcpy.ResetEnvironments()
arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = snap
arcpy.env.snapRaster = snap
arcpy.env.cellSize = snap
arcpy.env.compression = "LZW"
arcpy.env.mask = pyromes  # Otherwise it will extend into Canada

##-----create output folders
out = r"C:\Users\Charlie\Desktop\mosaic_test_data"
# function to create folders- creates folders if they don't already exist
def checkPath(path):
    try: os.makedirs(path)
    except: pass
# location for outputs
# output_folder = out
# checkPath(output_folder)
# mosaic folder
mosaic_folder = os.path.join(out, "Mosaic")
checkPath(mosaic_folder)
# mosiac folder- FLP folder
mosaic_flp_folder = os.path.join(mosaic_folder, "FLP")
checkPath(mosaic_flp_folder)
# backfill inputs folder
backfill_inputs_folder = os.path.join(out, "Backfill_Inputs")
checkPath(backfill_inputs_folder)
# backfill folder- masks folder
mask_folder = os.path.join(backfill_inputs_folder, "Masks")
checkPath(mask_folder)
# backfill folder- zonal folder
zone_folder = os.path.join(backfill_inputs_folder, "Zonal")
checkPath(zone_folder)
# final - clipped folder
final_folder = os.path.join(out, "Final")
checkPath(final_folder)

##-----final environment setup
arcpy.env.workspace = out
# set scratch workspace
arcpy.env.scratchWorkspace = r"C:\ArcTemp"

### Mosaic section

Although pyrome boundaries don't overlap, FSim simulates fires that can burn outside of a pyrome boundary. As a result, FSim outputs for bp and flp can overlap between pyromes. To account for this, we mosaic and sum the outputs for bp and flp, which requires recalculating bp and flp for the cells that overlap. The recalculation is an average of the overlapping cell values weighted by the number of FSim iterations in each overlapping pyrome. 

In [38]:
##-----general functions and set up

#--make a table view for getting the number of FSim iterations in each pyrome
arcpy.MakeTableView_management(in_table = iteration_lut_path, out_view = "lut")

#--retrieve pyrome strings in the lookup table as a list
# empty list
pyromes_list = []
# search cursor to iterate through the pyrome names and append them to the empty list
with arcpy.da.SearchCursor("lut", field_names = ["PYROME"]) as cursor:
    for row in cursor:
        pyromes_list.append(row[0])

#--function to get number of iterations for each pyrome
def get_itr(pyrome):
    """
    parameters:
    pyrome - pyrome name, passed as a string
    
    """
    with arcpy.da.SearchCursor("lut", field_names = ["PYROME", "ITR"]) as cursor:
        for row in cursor:
            if row[0] == pyrome:
                return row[1]

#--function to save raster outputs as tifs with compression
def save_tif(output, folder, custom_name = None):
    """
    parameters:
    output - output to be saved, passed as the variable name
    folder - directory to save output, passed as a variable or file path
    custom_name - optional, desired output name passed as a string
    
    """
    if custom_name:
        name = custom_name
    else:
        name = [var_name for var_name, obj in globals().items() if obj is output]
        name = str(name[0])
    output_path = os.path.join(folder, f"{name}.tif")
    try:
        arcpy.CopyRaster_management(output, output_path)
        print(f"Saved raster to: {output_path}")
    except Exception as e:
        print(f"Error saving raster: {e}")

In [39]:
##-----processing functions

#--extract bp raster from multiband FSim output
def get_bp_rasters(pyrome):
    """
    parameters:
    pyrome - pyrome name, passed as a string
    
    """
    r = Raster(os.path.join(fsim270, "{0}.tif".format(pyrome)))
    bp = ExtractBand(r, [1])
    return(bp)

#--create a raster of the number of times each pixel burned over the total number of FSim iterations
def get_times_burned(pyrome):
    """
    parameters:
    pyrome - pyrome name, passed as a string
    
    """
    r = Raster(os.path.join(fsim270, "{0}.tif".format(pyrome)))
    itr = get_itr(pyrome)
    bp = ExtractBand(r, [1])
    times_burned_in_pyrome = itr * bp
    return(times_burned_in_pyrome)

#--create a raster of the number of times each pixel burned over the specified flp class
def get_times_flp_burned(pyrome, band):
    """
    parameters:
    pyrome - pyrome name, passed as a string
    band - flp band, passed as an integer
    
    """
    r = Raster(os.path.join(fsim270, "{0}.tif".format(pyrome)))
    itr = get_itr(pyrome)
    bp = ExtractBand(r, [1])
    flp = ExtractBand(r, [band])
    times_flp_burned_in_pyrome = itr * bp * flp
    return(times_flp_burned_in_pyrome)

#--calculate final flp raster: number of times each pixel burned within flp class over total times burned
def x_flp_burned_over_total(flp, total):
    """
    parameters:
    flp - raster of number of times burned within flp class
    total - raster of total number of times burned over all FSim iterations
    
    """
    with arcpy.EnvManager(extent = "MAXOF"):
        final_flp = flp / total
        # fill nodata with 0
        final_flp0 = Con(IsNull(final_flp), 0, final_flp)
        return(final_flp0)

#--function to sum rasters, effectively mosaicking them
def sum_of_raster_list(this_list):
    """
    parameters:
    this_list - a list of rasters
    
    """
    with arcpy.EnvManager(extent = "MAXOF"):
        sum_of_rasters = arcpy.sa.CellStatistics(this_list, "SUM", "DATA")
        return(sum_of_rasters)

In [72]:
##-----run processing functions

#--map functions to pyrome outputs to get a mosaicked bp raster
# retrieves a list of bp rasters from every pyrome
list_of_bp_rasters = list(map(get_bp_rasters, pyromes_list))
# adds the bp rasters together
bp_270m = sum_of_raster_list(list_of_bp_rasters)
save_tif(bp_270m, mosaic_folder)

#--map functions to pyrome outputs to get a mosaicked raster of total times burned
# retrieves a list of rasters of total times burned from every pyrome
list_x_burned = list(map(get_times_burned, pyromes_list))
# adds the total times burned rasters together
total_times_burned = sum_of_raster_list(list_x_burned)
save_tif(total_times_burned, mosaic_folder)

#--loop through each flp class to create the final mosaicked flp rasters for each class
for i in range(1, 7):
    
    # name (eg, flp1)
    flp_name = f"flp{i}"
    print(flp_name)
    
    # band index (flp 1 is band 2, flp 2 is band 3, and so on)
    band_index = i + 1
    
    # get a list of rasters that show the number of times a pixel burned within flp class
    list_flp_burned = list(map(get_times_flp_burned, pyromes_list, repeat(band_index)))
    
    # take a sum of the raster list to get the total number of times a pixel burned within each flp class
    flp_times_burned = sum_of_raster_list(list_flp_burned)
    save_tif(flp_times_burned, mosaic_folder, custom_name = f"x_times_burned_in_{flp_name}")
    
    # calculate final flp- x times burned in flp / total times burned
    flp_270m = x_flp_burned_over_total(flp_times_burned, total_times_burned)
    save_tif(flp_270m, mosaic_flp_folder, custom_name = f"{flp_name}_270m")

Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\bp_270m.tif
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\total_times_burned.tif
flp1
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\x_times_burned_in_flp1.tif
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\FLP\flp1_270m.tif
flp2
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\x_times_burned_in_flp2.tif
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\FLP\flp2_270m.tif
flp3
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\x_times_burned_in_flp3.tif
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\FLP\flp3_270m.tif
flp4
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\x_times_burned_in_flp4.tif
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\FLP\flp4_270m.tif
flp5
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Mosaic\x_times_burned_in_flp5.tif
Saved r

### Backfill section

In order to arrive at the final bp and flp rasters, we backfill pixels that are burnable but didn't burn during the FSim runs with minimum bp and zonal flp values. Burnable pixels are any pixels with a burnable cell type in the input fuel model. For bp, burnable pixels that didn't burn are backfilled with the minimum bp value of 0.000008. For flp, burnable pixels that didn't burn are backfilled with a zonal average based on the likeliness of crown fire behavior. The value of the raster used to identify zones is the unique combinations of the fuel model categories, binary crown likely values, and pyrome identifiers.

In [73]:
# retrieve mosaicked bp and set extent
bp = os.path.join(mosaic_folder, "bp_270m.tif")
arcpy.env.extent = os.path.join(mosaic_folder, "bp_270m.tif")

##--create burnable mask from the fm40
print("starting burnable mask")
fm40_rast = arcpy.Raster(fm40)
# conditional that converts all burnable cell types to 1, and all other cell types to 0
burnable_mask = Con((fm40_rast > 99) & (fm40_rast < 205), 1, 0)
save_tif(burnable_mask, mask_folder)

##--create the crown-fire-likely mask
print("starting crown fire likeliness mask")
cc_rast = arcpy.Raster(cc)
cbh_rast = arcpy.Raster(cbh)
crownlikely_mask = Con((cc_rast > 0) & (cbh_rast <= 30), 1, 0)
save_tif(crownlikely_mask, mask_folder)

##--find combinations of fm40, crown-fire-likely, and pyrome identifiers to serve as similar zones during flp backfill
print("starting flp zones")
crownlikely_rast = arcpy.Raster(os.path.join(mask_folder, "crownlikely_mask.tif"))
pyromes_rast = arcpy.Raster(pyromes)
flp_zones = arcpy.sa.Combine([fm40_rast, crownlikely_mask, pyromes_rast])
save_tif(flp_zones, mask_folder)

##--identify pixels that need to be backfilled (burnable but didn't burn)
print("starting mask of pixels to backfill")
bp_rast = arcpy.Raster(bp)
burnable_noburn_mask = Con((bp_rast == 0) & (burnable_mask == 1), 1, 0)
save_tif(burnable_noburn_mask, mask_folder)

##--mask for pixels that did burn
print("starting mask of pixels that burned")
has_bp = Con(bp_rast > 0, 1, 0)
save_tif(has_bp, mask_folder)

starting burnable mask
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Masks\burnable_mask.tif
starting crown fire likeliness mask
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Masks\crownlikely_mask.tif
starting fm40 and crown fire mask
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Masks\fm_crownfire_mask.tif
starting fm40-crown fire-pyromes mask
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Masks\fm_crownfire_pyrome.tif
starting mask of pixels to backfill
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Masks\burnable_noburn_mask.tif
starting mask of pixels that burned
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Masks\has_bp.tif


In [41]:
#-----backfill flp rasters
arcpy.env.workspace = mosaic_flp_folder
flp_list = arcpy.ListRasters("*flp*")

#--calculate zonal average on flp, backfill pixels, and clip flp to the wcs landscape buffers
for flp in flp_list:
    
    # string slice to remove .tif extension on flp name
    flp_name = flp[0:-4]
    print(f"starting zonal average on {flp_name}")
    
    # extract values to summarize
    flp_extr = SetNull(has_bp, flp, "VALUE = 0")
    
    # run zonal statistics tool
    flp_avg = ZonalStatistics(flp_zones, "VALUE", flp_extr, "MEAN", "DATA")
    
    # export result
    save_tif(flp_avg, zone_folder, custom_name = f"{flp_name}_zoneAvg")
    
    with arcpy.EnvManager(mask = wcs_buffer, extent = wcs_buffer):
        print(f"starting backfill on {flp_name}")
        
        # set pixels to zone average if burnable but not burned, otherwise keep FLP values
        backfilled_flp = Con(burnable_noburn_mask == 1, flp_avg, flp)
        
        # save output
        save_tif(backfilled_flp, final_folder, custom_name = f"{flp_name}_backfilled")

starting zonal average on flp1_270m
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Zonal\flp1_270m_zoneAvg.tif
starting backfill on flp1_270m
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Final\flp1_270m_backfilled.tif
starting zonal average on flp2_270m
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Zonal\flp2_270m_zoneAvg.tif
starting backfill on flp2_270m
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Final\flp2_270m_backfilled.tif
starting zonal average on flp3_270m
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Zonal\flp3_270m_zoneAvg.tif
starting backfill on flp3_270m
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Final\flp3_270m_backfilled.tif
starting zonal average on flp4_270m
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Backfill_Inputs\Zonal\flp4_270m_zoneAvg.tif
starting backfill on flp4_270m
Saved raster to: C:\Users\Charlie\Desktop\mosaic_t

In [56]:
#------backfill bp
bp_fill_value = 0.000008
print("starting backfill for bp")
with arcpy.EnvManager(mask = wcs_buffer, extent = wcs_buffer):
    
    # set pixels to the bp fill value if they are burnable pixels that didn't burn, otherwise keep the burn probability value
    bp_270m_backfilled = Con(burnable_noburn_mask == 1, bp_fill_value, bp)
    
    # save the output
    save_tif(bp_270m_backfilled, final_folder)

starting backfill for bp
Saved raster to: C:\Users\Charlie\Desktop\mosaic_test_data\Final\bp_270m_backfilled.tif


### Building in checks

1. All of the final flp rasters should sum to 1 or 0:

All of the pixels in the sum of the flp rasters should have a value of 0 or 1. Due to rounding during the process of creating them, the flp sum raster will likely also have values very close to 1

In [55]:
##-----check 1
import numpy as np

#--all of the final flp rasters should sum to 1 or 0
# create a list of final backfilled flp rasters
arcpy.env.workspace = final_folder
flp_list = arcpy.ListRasters("*flp*")

# add the 6 rasters together
with arcpy.EnvManager(mask = wcs_buffer, extent = wcs_buffer):
    
    flp_sum = arcpy.sa.CellStatistics(flp_list, "SUM", "DATA")
    
# convert flp sum raster into an array and find unique values
flp_arr = arcpy.RasterToNumPyArray(flp_sum)
values = np.unique(flp_arr)
print(f"Unique values in flp sum raster: {values}")

Unique values in flp sum raster: [-3.4028235e+38  0.0000000e+00  9.9999970e-01  9.9999976e-01
  9.9999982e-01  9.9999988e-01  9.9999994e-01  1.0000000e+00
  1.0000001e+00  1.0000002e+00  1.0000004e+00]


2. Where there are bp values, there should also be flp values:

The number of cells that have a bp value should match the number of cells that have an flp value 

In [57]:
##-----check 2
# create binary rasters for flp sum and bp where a value of 1 is a cell with a value greater than 0 
flp_sum_con = Con(flp_sum > 0, 1, 0)
bp_con = Con(bp_270m_backfilled > 0, 1, 0)

# convert to arrays
flp_con_arr = arcpy.RasterToNumPyArray(flp_sum_con)
bp_con = arcpy.RasterToNumPyArray(bp_con)

# the number of cells with values greater than 0 should match in each
flp_sum_val = np.sum(flp_con_arr)
bp_val = np.sum(bp_con)
matches = flp_sum_val == bp_val
print(f"The number of cells where there is a bp value and an flp value is equal: {matches}")

The number of cells where there is a bp value and an flp value is equal: True
